Code for Sequence Alignment, Sequence Similarity, and calculating the distance matrix from these

In [1]:
import scipy.io as sio
#seq_data = sio.loadmat('flhivdata.mat')

In [2]:
#seq_data['lc5'] # example sequence from dataset

In [9]:
align = ['AAGCCGCACACAGACCCTGAG', 
          'AAGCTGCACGCAGACCCTGAG', 
          'AGGCTGCACGCAGACCCTGAG', 
          'AAGCTGCACGTGGACCCTGAG', 
          'AGGCTGCACGTGGACCCTGAG', 
          'AGGCTGCACGTGGACCCTGAG', 
          'AAGCTGCATGTGGACCCTGAG']

Below is a similarity matrix determined by the likelihood of transition vs transversion mutations. Over the entire genome, a base that is related by a transition to another base is twice as likely to be mutated to that base than a base it is related to by transversion. For example, A and G are related by transition, since they are both purines. A transition mutation from A to G or vice-versa would then be twice as likely as a transversion mutation. Thus, in a similarity matrix, we could define an A to G mutation or a G to A transition as half as dissimilar as any transversion mutation. We could also define the lack of mutation, in which a base stays the same, as a score of 0. Scores in this similarity matrix range from identical at 0 to very dissimilar at 2.

In [10]:
DNA_sim = {'G': { 'A': 1, 'G': 0, 'C': 2, 'T': 2 },
         'C': { 'A': 1, 'G': 2, 'C': 0, 'T': 1 },
         'T': { 'A': 2, 'G': 2, 'C': 1, 'T': 0 },
         'A': { 'A': 0, 'G': 1, 'C': 2, 'T': 2 }}

In [11]:
def sequenceAlign(s1, s2, simMatrix=DNA_sim, insert=8, extend=4):

  numI = len(s1) + 1
  numJ = len(s2) + 1

  scoreMatrix = [[0] * numJ for x in range(numI)]
  routeMatrix = [[0] * numJ for x in range(numI)]
  
  for i in range(1, numI):
    routeMatrix[i][0] = 1
  
  for j in range(1, numJ):
    routeMatrix[0][j] = 2
  
  for i in range(1, numI):
    for j in range(1, numJ):
    
      penalty1 = insert
      penalty2 = insert
      
      if routeMatrix[i-1][j] == 1:
        penalty1 = extend
        
      elif routeMatrix[i][j-1] == 2:
        penalty2 = extend
        
      similarity = simMatrix[ s1[i-1] ][ s2[j-1] ]
      
      paths = [scoreMatrix[i-1][j-1] + similarity, # Route 0
               scoreMatrix[i-1][j] - penalty1, # Route 1
               scoreMatrix[i][j-1] - penalty2] # Route 2                     
      
      best = max(paths)
      route = paths.index(best)           

      scoreMatrix[i][j] = best
      routeMatrix[i][j] = route
      
  alignA = []
  alignB = []
  
  i = numI-1
  j = numJ-1
  score = scoreMatrix[i][j]

    
  while i > 0 or j > 0:
    route = routeMatrix[i][j]    

    if route == 0: # Diagonal
      alignA.append( s1[i-1] )
      alignB.append( s2[j-1] )
      i -= 1
      j -= 1
  
  alignA.reverse()
  alignB.reverse()
  alignA = ''.join(alignA)
  alignB = ''.join(alignB)

  return score, alignA, alignB 

In [12]:
def calcSeqSimilarity(s1, s2, simMatrix):
    numPlaces = min(len(s1), len(s2))
    totalScore = 0.0
    for i in range(numPlaces):
        residueA = s1[1]
        residueB = s2[i]
        totalScore += simMatrix[residueA][residueB]
    return totalScore

In [13]:
def getDistanceMatrix(seq_data, simMatrix):
    n = len(seq_data)
    matrix = [[0.0] * n for x in range(n)]
    maxScores = [calcSeqSimilarity(x, x, simMatrix) for x in seq_data]
    
    for i in range(n-1):
        s1 = seq_data[i]
        
        for j in range(i+1, n):
            s2 = seq_data[j]
            
            score, alignA, alignB = sequenceAlign(s1, s2, simMatrix, insert=8, extend=4)
            maxScore = max(maxScores[i], maxScores[j])
            dist = maxScore - score
            
            matrix[i][j] = dist
            matrix[j][i] = dist
            
    return matrix

In [ ]:
dMatrix = getDistanceMatrix(align, DNA_sim)